In [2]:
import requests, time
from bs4 import BeautifulSoup

def get_news_list(root, url, date_boundary):
    article_list = []
    while 1:
        path = root + url
        html = requests.get(path)
        soup = BeautifulSoup(html.text)
        body = soup.find('ul', 'wp_article_list')  # 寻找对应的新闻列表
        for item in body.find_all('li'):  # 遍历新闻列表中的每条新闻组
            date = item.find('span', 'Article_PublishDate').string  # 通过每条新闻组中 	span的class获取时间字符串（不用.string获取的为tag）
            convert_date = time.mktime(time.strptime(date, "%Y-%m-%d"))  # 用秒数来表示时间
            if convert_date < date_boundary:
                break
            title = item.find('a')['title']
            href = item.find('a')['href']
            article_list.append([date, title, href])  # 将数据追加保存到article_list中
        # print(title, href ,date)
        if time.mktime(time.strptime(date, '%Y-%m-%d')) > date_boundary:
            next_page = soup.find('ul', 'wp_paging clearfix')
            url = next_page.find('a', {'class': 'next'})['href']
        else:
            break
    #print(article_list)
    return article_list

In [11]:
def crawl_news(root,article_list):
    news_dict = {}
    i = 1
    for item in article_list:
        date,title,href = item # 将数组中的三个元素分别赋值给变量
        #print(date,title,href)
        if 'http' in href:
            path = href
        else:
            path = root + href
        f = requests.get(path)
        soup = BeautifulSoup(f.text)
        body = soup.find('div',{'class','acd'})
        content = body.find('div',{'class','wp_articlecontent'})
        text = u''
        for a in content.strings: # 效果等同于text = content.text
            text += a
        
        publis = body.find_all('span','arti_publisher')[1].string
        department = publis.split(u'\uff1a')[1] # 将publis通过 ':'进行分割，效果等同于 publis.split(':')
        if department == '':
            department = 'none'
        view_times = body.find('span',['class','WP_VisitCount']).text
        #逐个把新闻信息加入字典
        news_dict[i] = {'date':date,'title':title,'source':department,'content':text,'views':view_times,'url':path}
        
        time.sleep(0.1) # 控制爬虫速度
        i += 1;
    return news_dict

def search(keywords,news_dict):
    result = {} # result为字典数据类型
    title_list = [] # 列表数据类型
    source_list = []
    content_list = []
    for x in news_dict: # 对于字典数据类型，x的值为1、2、3......
        if keywords in news_dict[x]['title']:
            title_list.append([x,news_dict[x]['date'],news_dict[x]['title']])
        if keywords in news_dict[x]['source']:
            source_list.append([x,news_dict[x]['date'],news_dict[x]['source']])
        if keywords in news_dict[x]['content']:
            content_list.append([x,news_dict[x]['date'],news_dict[x]['content']])
    
    result['title'] = title_list
    result['source'] = source_list
    result['content_list'] = content_list
    return result
        # print(x)

In [45]:
if __name__ == '__main__': # 如果这个module文件是被当成程序来执行，那么，该__name__属性的值就是"__main__"
    root = "http://www.suibe.edu.cn/_s19"
    url = "/1416/list.psp"
    date_boundary = time.mktime(time.strptime("2019-08-01", "%Y-%m-%d"))
    news_list = get_news_list(root, url, date_boundary)
    news_dict = crawl_news(root,news_list)
    result = search(u'上海对外经贸大学专场',news_dict)
    for x in result:
        print(x[0],x[1])
        print(x[2])

ConnectionError: HTTPConnectionPool(host='www.suibe.edu.cn', port=80): Max retries exceeded with url: /_s19/1416/list.psp (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0664FD50>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

In [43]:
result = search(u'上海对外经贸大学专场',news_dict)
print(result)

{'title': [], 'source': [], 'content_list': [[2, '2019-09-24', '9月19日下午，陆家嘴金融城2019秋季名校直通车招聘会(上海对外经贸大学专场)在松江校区信息楼二楼大厅举行。来自银行、保险、证券、基金、会计等行业的29家企业参加了本次招聘会，参会企业向毕业生提供了近600余个招聘岗位。我校学生和来自上海各高校的近400名毕业生参加此次招聘会。招聘会现场气氛热烈，毕业生们手持简历在各招聘展位前有序排队投递简历，并与企业招聘人员进行交流互动。\xa0\xa0本次招聘会得到了上海陆家嘴金融城人力资源管理联合会、陆家嘴人才金港的大力支持。在多方合作下，今年秋季专场招聘会参会企业的行业属性和招聘岗位与学校的学科专业匹配度进一步提高，为我校毕业生提供了良好的求职选择机会。十月份中下旬还将举办“前程无忧2019秋季名校直通车招聘会”(上海对外经贸大学专场)、“2019易校招优质企业招聘会”(上海对外经贸大学专场)、“2019上海保险专场校园招聘会”(上海对外经贸大学专场)。\xa0\xa0\xa0']]}
